In [1]:
import warnings
warnings.filterwarnings('ignore')

# Scrapping Digital Library ITB (3.0)

## Pendahuluan

Banyak hal yang terjadi sejak saya melakukan *scraping* website Digital Library ITB (Digilib ITB). Beberapa diantaranya tentang perbaikan website (yang tidak melempar *error* ketika melakukan pencarian data buku), dan adanya akses katalog buku menggunakan protokol OAI-PMH (Open Archive Initiative - Protocol for Metadata Harvesting). Secara sederhana, protokol ini memberikan data XML -- jauh lebih mudah diproses menjadi `dict` ketimbang HTML -- saat kita melakukan *scrapping*. Sebagai contoh, lihat tampilan data di [link ini](https://digilib.itb.ac.id/oai-ristek.php?verb=GetRecord&identifier=12345&metadataPrefix=oai_dc).

## Scrapping
Untuk melakukan *scrapping* kali ini, kita membutuhkan module [`xmltodict`](https://github.com/martinblech/xmltodict) yang memudahkan mengubah data XML menjadi `dict`, dan tentu saja module `request`.

In [2]:
from xmltodict import parse
from urllib.request import urlopen

Ada baiknya kita menaruh URL yang akan digunakan dalam scrapping, dan item apa saja yang akan kita ambil.

In [3]:
URL   = "https://digilib.itb.ac.id/oai-ristek.php?verb=GetRecord&identifier={}&metadataPrefix=oai_dc" 
items = ['dc:title', 'dc:creator', 'dc:contributor', 'dc:nidn', 'dc:subject', 'dc:date',
         'dc:language', 'dc:type', 'dc:Keyword', 'dc:Institution', 'dc:description']

Lalu data buku dapat diambil dengan mudah, salah satunya dengan langkah berikut:

In [4]:
def dictify(num):
    result = {}
    
    # akses data XML
    with urlopen(URL.format(num)) as f:
        raw = f.read().decode('utf8', 'ignore')
    
    # lokasi data yang kita inginkan
    data = parse(raw)['OAI-PMH']['GetRecord']['record']['metadata']['oai_dc:dc']

    # catat data
    for i in items:
        result[i] = data[i]
    
    return result

Berikut adalah contoh hasil dari fungsi di atas

In [5]:
#dictify(12345)

## Mendownload Data

Saat ini, ada 47536 data buku yang tersimpan di Digilib ITB. Dalam keadaan seperti ini, saya lebih suka melakukannya secara *batch*. Sebagai contoh, saya mengambil 1000 data, lalu menyimpannya sebagai sebuah `pickle`. Hal ini menguntungkan karena jika ada *error* dalam proses *download*, saya tidak perlu mengulang proses dari awal lagi (duh).

Saya memerlukan dua module dalam proses ini. Sebenarnya hanya satu, namun module [`tqdm`](https://github.com/tqdm/tqdm) sangat berguna untuk mengetahui sudah sejauh mana proses *download* telah berjalan :D

In [6]:
import pickle
from tqdm import tqdm

In [7]:
def batch_download(start_index, end_index, batch_size=100):
    """
    Mengambil data dari start_index sampai end_index.
    Menyimpan setiap batch_size data sebagai sebuah pickle.
    """
    
    for batch in range(start_index, end_index+1, batch_size):
        
        # ambil data secara batch
        result = {}
        for num in tqdm(range(batch, batch+batch_size), desc=str(batch)):
            tmp = str(num)
            result[tmp] = dictify(tmp)

        # simpan sebagai sebuah pickle
        with open('pickle_{}'.format(batch), 'wb') as outfile:
            pickle.dump(result, outfile, -1)

In [8]:
# Proses ini memerlukan sekitar 4.8 jam
#batch_download(9701, 47600)    # 20 Juli 2020
#batch_download(47501, 51900)   # 1 Desember 2020

## Bentuk DataFrame

Selanjutnya, kita perlu menyimpan data dalam format yang mudah digunakan. Saya menggunakan `DataFrame`, karena kemampuannya dalam memroses data dan kemampuan *import-export* yang dimiliki

In [9]:
import pandas as pd

Selanjutnya kita baca data yang kita miliki

In [10]:
result = {}
for n in tqdm(range(1, 51900, 100)):
    with open('pickle/pickle_{}'.format(n), 'rb') as outfile:
        result.update(pickle.load(outfile))

100%|██████████| 519/519 [00:02<00:00, 184.29it/s]


Selanjutnya, kita akan membentuk `DataFrame` dengan cara

In [11]:
data = pd.DataFrame.from_dict(result, orient='index')

Namun, beberapa baris data punya data yang hilang. Dengan berpikir se-konservasif mungkin, mari hapus baris yang tidak mengandung sama sekali judul

In [12]:
data.dropna(subset=['dc:title'], inplace=True)

In [13]:
data.tail()

,dc:title,dc:creator,dc:contributor,dc:nidn,dc:subject,dc:date,dc:language,dc:type,dc:Keyword,dc:Institution,dc:description
51861,PELACAKAN OBJEK DAN PENGAMBIL GAMBAR WAJAH PAD...,"Dasa Nanda Putra, Rafi","Dr. Ir. Mervin Tangguar Hutabarat, M.Sc.;Muham...",0018056501;0023108802,STEI - Teknik Elektro,2020-11-03 13:32:09,Indonesia,Thesis,"Kriminalitas, Pencacahan, Pendeteksian, Pengen...",INSTITUTE TEKNOLOGI BANDUNG,Tingginya angka kriminalitas di Indonesia teru...
51862,METODE KETERLIBATAN STAKEHOLDER UNTUK IMPLEMEN...,"Maheswari, Hesti","Ir. Gatot Yudoko, M.A.Sc., Ph.D.;Akbar Adhi Ut...",0025076102;0005067707,SBM - Sains Manajemen,2020-11-20 14:48:01,Indonesia,Thesis,"metode keterlibatan, sustainable reverse logis...",INSTITUTE TEKNOLOGI BANDUNG,Pengolahan sampah elektronik di Indonesia dido...
51863,EVALUASI TARIF PDAM KOTA BANDUNG BERDASARKAN P...,"Ajeng Koesoemo Roekmi, Raden","Ir. Widiarto, MCRP., Ph.D;",None,SAPPK - Perencanaan Wilayah dan Kota,2020-11-21 16:18:38,Indonesia,Thesis,"Prinsip Keterjangkauan, Prinsip Keadilan, Stan...",INSTITUTE TEKNOLOGI BANDUNG,Penyediaan air bersih oleh Pemerintah Daerah h...
51864,DAMPAK KINERJA USAHA MIKRO KECIL DAN MENENGAH ...,"Ahmad Muzakky, Faqih","Dr. Ir. Subiakto, M.B.A.;",0014046017,SBM - Sains Manajemen,2020-11-24 13:36:51,Indonesia,Thesis,"inklusi keuangan, UMKM, e-commerce",INSTITUTE TEKNOLOGI BANDUNG,Sebuah pertanyaan telah banyak diangkat dalam ...
51865,USULAN STRATEGI USAHA PERUSAHAAN KONSULTAN ITB...,"Fitriani, Novi","Deddy Priatmodjo Koesrindartoto, Ph.D.;",0003047006,Magister Bisnis dan Administrasi - Teknologi,2020-11-27 15:23:57,Indonesia,Thesis,"Strategi Bisnis, Akuisisi, Peningkatan Keuanga...",INSTITUTE TEKNOLOGI BANDUNG,Institut Teknologi Bandung (ITB) merupakan per...


selanjutnya mari simpan data sebagai TSV (Tab Separated Value)

In [ ]:
data.to_csv('raw_digilib.tsv', sep='\t')